# Cases in Malaysia
> Updates on the respiratory illness that has infected more than one million people and killed tens of thousands.

- toc:false
- branch: master
- badges: false
- hide_colab_badge: true
- comments: false

## COVID-19: Malaysia at a Glance

In [0]:
#hide
import pandas as pd
import numpy as np
import altair as alt
#hide
# COVID-19 Data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'

# confirmed, deaths, recovered
def get_time_series(type):
  _url = f'{base_url}/csse_covid_19_time_series/time_series_covid19_{type}_global.csv'
  _dff = pd.read_csv(_url)
  _cols = _dff.columns[~_dff.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
  _dff = (_dff.groupby('Country/Region')[_cols].sum().stack().reset_index(name = 'Cases')
        .rename(columns = {'level_1': 'Date', 'Country/Region': 'Country'}))
  _dff['Date'] = pd.to_datetime(_dff['Date'], format='%m/%d/%y')
  return _dff

# eg. 03-27-2020
def get_df_daily(date):
  _url = f'{base_url}/csse_covid_19_daily_reports/{date}.csv'
  return pd.read_csv(_url)

df_confirmed = get_time_series('confirmed')
df_recovered = get_time_series('recovered')
df_deaths = get_time_series('deaths')

date_latest = df_confirmed['Date'].max()

def countries_cases(df, n=5000, date_as_of=date_latest):
  df_latest = df[df['Date'].eq(date_as_of)]
  df_latest_n = df_latest[df_latest['Cases'] > n]
  return df_latest_n

# Last n days of type e.g. confirmed, recovered, deaths
df_last = lambda name, _df, n: _df[_df['Date'].isin(_df['Date'].tail(n))].set_index(['Country', 'Date']).rename(columns={'Cases': name})

# Last 2 days all global
df_summary = lambda last_n: pd.concat([df_last('Confirmed', df_confirmed, last_n), df_last('Deaths', df_deaths, last_n), df_last('Recovered', df_recovered, last_n)], axis=1)

#df_summary.xs('Malaysia', level = 0)

df_global = df_summary(2).groupby(['Date']).sum().reset_index()

df_global_summary = df_global.stack().reset_index().rename(columns={'level_1': 'Type', 0: 'Cases'})

#based_chart = alt.Chart(df_global).encode(x = 'monthdate(Date):O')
#closed_chart = based_chart.mark_bar().encode(y = 'Deaths:Q')
#confirmed_chart = based_chart.mark_bar().encode(y = 'Confirmed:Q')

#print(date_latest)
#hide
def chart_countries_timeline(_df, countries, since_date='1/23/2020'):
  _df = _df[_df['Country'].isin(countries)]
  _df = _df[_df['Date'].ge(since_date)]
  selection = alt.selection_multi(fields=['Country'], bind='legend')
  chart = alt.Chart(_df).mark_line().encode(
          x='Date:T', 
          y='Cases:Q',
          #y=alt.Y('Cases:Q', scale=alt.Scale(type='log')),
          color='Country',
          tooltip=list(_df),
          opacity=alt.condition(selection,alt.value(1), alt.value(0.2))
        ).properties(width=1000).add_selection(selection)
  return chart

def chart_countries_timeline_diff(df, country):
  _df = df[df['Country'] == country]
  _df = _df[['Date', 'Cases']].set_index('Date')
  source = _df.diff().reset_index()
  chart = alt.Chart(source).mark_bar().encode(
              x='Date:T',
              y='Cases:Q' #y=alt.Y('Cases:Q', scale=alt.Scale(type='log'))
          )
  return chart

def chart_countries_cases(df, n, date_as_of=date_latest):
    bars = alt.Chart(countries_cases(df, n, date_as_of)).mark_bar().encode(
        x=alt.X('Country:O', sort='-y'),
        y='Cases:Q'
    )

    text = bars.mark_text(
        align='center',
        baseline='bottom'
    ).encode(
        text='Cases:Q'
    )
    return (bars + text).properties(width=500)

s_date_latest = date_latest.strftime('%m/%d/%Y')
n_confirmed = df_global.loc[1]['Confirmed']
n_deaths = df_global.loc[1]['Deaths']
n_recovered = df_global.loc[1]['Recovered']

df_global_diff = df_global.diff()
n_confirmed_diff = df_global_diff.loc[1]['Confirmed']
n_deaths_diff = df_global_diff.loc[1]['Deaths']
n_recovered_diff = df_global_diff.loc[1]['Recovered']

weekDays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday") 

In [3]:
#hide_input
df_confirmed_my = df_confirmed[df_confirmed['Country'] == 'Malaysia'].rename(columns={'Cases': 'Total number of Cases', 'Date': 'Reported Date'})

alt.Chart(df_confirmed_my).mark_bar().encode(
    x='Reported Date:T',
    y='Total number of Cases:Q'
).properties(
    width=600,
    title='Cummulative total number of COVID-19 cases in Malaysia'
)

alt.Chart(...)

In [12]:
#hide_input
df_confirmed_my[['Reported Date', 'Total number of Cases']].set_index('Reported Date').rename(columns={'Total number of Cases': 'Total'}).transpose()

Reported Date,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05
Total,0,0,0,3,4,4,4,7,8,8,8,8,8,10,12,12,12,16,16,18,18,18,19,19,22,22,22,22,22,22,22,22,22,22,22,22,23,23,25,29,29,36,50,50,83,93,99,117,129,149,149,197,238,428,566,673,790,900,1030,1183,1306,1518,1624,1796,2031,2161,2320,2470,2626,2766,2908,3116,3333,3483,3662
